In [1]:
!pip install -U transformers --quiet
!pip install torch
!git clone https://github.com/cardiffnlp/tweeteval /tmp/tweeteval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path '/tmp/tweeteval' already exists and is not an empty directory.


In [2]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [3]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

[{'score': 0.16661465167999268,
  'sequence': 'Send these pictures back!',
  'token': 3493,
  'token_str': ' pictures'},
 {'score': 0.10792841017246246,
  'sequence': 'Send these photos back!',
  'token': 2356,
  'token_str': ' photos'},
 {'score': 0.07670916616916656,
  'sequence': 'Send these emails back!',
  'token': 5575,
  'token_str': ' emails'},
 {'score': 0.04860782250761986,
  'sequence': 'Send these images back!',
  'token': 3156,
  'token_str': ' images'},
 {'score': 0.048417482525110245,
  'sequence': 'Send these letters back!',
  'token': 5430,
  'token_str': ' letters'}]

In [4]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/tmp/tweeteval/datasets/hate/train_text.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [5]:
print(dataset[:5])

[{'input_ids': tensor([    0,  1039, 12105,  2579,    92, 25152,     4,  3945,    47,    45,
         2273,    30, 10841,   459, 34865,   111,  5827, 38561,  8817,  3977,
         1657,  4950,  1295,    15,    47,  1174,  1437,     2])}, {'input_ids': tensor([    0,   250,   693,    54,    47, 42647,  1533,   498,   584, 25610,
        38594,   650,    16,    10, 19389,    47,   216,  1717,   478,    14,
         1514, 17841, 12736,  1437,     2])}, {'input_ids': tensor([    0,  1039, 12105,   787, 12105,   588,  1067,   109,    47,    33,
         2473,    50,    58,    51, 34348,  4462,    66,    30,    10,  5345,
          506, 11797,   242,   116,  1437,     2])}, {'input_ids': tensor([    0, 16625,  6096,   356,   179,    23,   162,   101,    10,   333,
          324,    11,    42, 32594,   328,  1437,     2])}, {'input_ids': tensor([    0, 30811,  3121,  3569,   693,   101,   787, 12105,  1437,     2])}]


In [6]:
!head /tmp/tweeteval/datasets/hate/train_text.txt

@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you… 
A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎 
@user @user real talk do you have eyes or were they gouged out by a rapefugee? 
your girlfriend lookin at me like a groupie in this bitch! 
Hysterical woman like @user 
Me flirting- So tell me about your father... 
The Philippine Catholic bishops' work for migrant workers should focus on families who are "paying the great... 
I AM NOT GOING AFTER YOUR EX BF YOU LIEING SACK OF SHIT ! I'm done with you dude that's why I dumped your ass cause your a lieing 😂😡 bitch 
When cuffin season is finally over 
Send home migrants not in need of protection, Peter Dutton tells UN, HEY DUTTON HOW ABOUT THE ONES THAT HAVE STAYED AND NOT LEFT THE COUNTRY WHEN THEY SHOULD OVERSTAYERS ? WHY DONT YOU GO AND ROUND ALL THEM UP ?  


In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

In [9]:
trainer.train()

trainer.save_model("./roberta-retrained")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8993
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1410


Step,Training Loss
500,2.415000
1000,2.121000


Saving model checkpoint to ./roberta-retrained/checkpoint-500
Configuration saved in ./roberta-retrained/checkpoint-500/config.json
Model weights saved in ./roberta-retrained/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./roberta-retrained/checkpoint-1000
Configuration saved in ./roberta-retrained/checkpoint-1000/config.json
Model weights saved in ./roberta-retrained/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./roberta-retrained
Configuration saved in ./roberta-retrained/config.json
Model weights saved in ./roberta-retrained/pytorch_model.bin


In [11]:
fill_mask = pipeline(
    "fill-mask",
    model="./roberta-retrained",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

loading configuration file ./roberta-retrained/config.json
Model config RobertaConfig {
  "_name_or_path": "./roberta-retrained",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file ./roberta-retrained/config.json
Model config RobertaConfig {
  "_name_or_path": "./roberta-retrained",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_i

[{'score': 0.19816094636917114,
  'sequence': 'Send these guys back!',
  'token': 1669,
  'token_str': ' guys'},
 {'score': 0.07832145690917969,
  'sequence': 'Send these people back!',
  'token': 82,
  'token_str': ' people'},
 {'score': 0.06482713669538498,
  'sequence': 'Send these refugees back!',
  'token': 4498,
  'token_str': ' refugees'},
 {'score': 0.052080925554037094,
  'sequence': 'Send these men back!',
  'token': 604,
  'token_str': ' men'},
 {'score': 0.038909319788217545,
  'sequence': 'Send these women back!',
  'token': 390,
  'token_str': ' women'}]